# Always-On NER Baseline

This notebook implements the "always-on" baseline for near real-time Named Entity Recognition (NER). In this approach, we run NER on every token as it arrives, simulating a scenario where we have no selective inference strategy.

This is the most computationally expensive approach, but it gives us the earliest possible detection of entities.

## 1. Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [10]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [11]:
from datasets import load_dataset

## 2. Load OntoNotes Dataset

We'll use the OntoNotes 5.0 dataset, which contains texts from various genres with named entity annotations.

In [12]:
# Load the English portion of OntoNotes 5.0
ontonotes = load_dataset("conll2012_ontonotesv5", "english_v12")
print(f"Dataset loaded with splits: {ontonotes.keys()}")

# Get basic statistics for each split
for split_name in ontonotes.keys():
    print(f"{split_name.capitalize()} set: {len(ontonotes[split_name])} examples")

Dataset loaded with splits: dict_keys(['train', 'validation', 'test'])
Train set: 10539 examples
Validation set: 1370 examples
Test set: 1200 examples
